<a href="https://colab.research.google.com/github/alexjochs/ECE_539_Penguins/blob/preprocess/PenguinsPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from matplotlib import image
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
from scipy.spatial import KDTree
import h5py

In [17]:
gdrive_data_filepath = r'/content/drive/MyDrive/Colab\ Notebooks/539\ Project/data'

In [18]:
VM_ROOT = r'/content'
cwd = None
def get_new_batch(target=None, load_all=False):
    assert target == None or load_all == False, f"can't get specific folder: {target} and load all data"
    if not target:
        print("Specifiy target")
    tgz_name = target + '.tgz'
    folder_path = os.path.join(gdrive_data_filepath, tgz_name)
    !tar --gunzip --extract --verbose --file={folder_path} --directory {VM_ROOT}
    cwd = os.path.join(VM_ROOT, target)

def save_batch_to_drive(target=None, save_all=False):
    # take working files, save them back to Gdrive
    assert target == None or save_all == False, f"can't get specific folder: {target} and load all data"
    tgz_name = target + '_gt.tgz'
    folder_path = os.path.join(gdrive_data_filepath, tgz_name)
    !tar -c -f -z -v {tgz_name} {folder_path}

In [19]:
annotations_path = VM_ROOT + r'/CompleteAnnotations_2016-07-11'

def run_all():
    json_filepath_list = get_json_files_from_folder()
    for filepath in json_filepath_list:
        df = run(filepath_=filepath)
        save_df_as_json(df)

def run(filepath_=None, target=None):
    assert filepath_ is not None, "at this time run() method needs a full filepath"
    data_group_name = filepath_[-10:-5]
    df = load_json_as_df(filepath_)
    df.loc[df.xy.isnull(), 'xy'] = [[]]
    # have to check if inner list has na values as well :/
    df_xy_ = to_1D(df['xy'])
    if df_xy_.isna().sum() > 0:
        df_xy_na_mask = df_xy_.isna()
        df.loc[df_xy_na_mask, 'xy'] = [[]]
    return df

def save_df_as_json(df):
    data_group_filename = df['imName'].iloc[0][:5] + '.json'
    try:
        os.mkdir('/content/annotations')
    except FileExistsError as e:
        print('looks like local annotations folder already exists!')
    print(data_group_filename)
    with open(os.path.join('/content/annotations', data_group_filename), 'w') as json_file:
        json.dump(json.loads(df.to_json(orient='records')), json_file)

def get_json_files_from_folder():
    json_filepath_list = []
    for filename in os.listdir(annotations_path):
        f = os.path.join(annotations_path, filename)
        if os.path.isfile(f):
            file_extension = os.path.splitext(f)[1]
        if file_extension == '.json':
            json_filepath_list.append(f)
    return json_filepath_list

def load_json_as_df(filepath):
    with open(filepath,'r') as json_file:
        json_data = json.loads(json_file.read())
    return pd.json_normalize(json_data, record_path =['dots'])

def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [27]:
# dots: 2d list of xy dots from users
# return longest (ie most penguins clicked) 1d list
def get_longest_dot_list(dots):
    if len(dots) == 0:
        return []
    if not type(dots[0]) == type([]):
        return []
    idx = 0
    if len(dots) > 1:
        for i, x in enumerate(dots):
            if len(x) > len(dots[idx]):
                idx = i
    return dots[idx]

# One hot encoding of user clicks
def make_sparse_mat(img_shape, dots):
    mat = np.zeros((img_shape[0], img_shape[1]))
    for dot in dots:
        mat[dot[1], dot[0]] = 1

    return mat

def gaussian_filter_density(gt):
    #Generates a density map using Gaussian filter transformation
    
    density = np.zeros(gt.shape, dtype=np.float32)
    
    gt_count = np.count_nonzero(gt)
    
    if gt_count == 0:
        return density

    # FInd out the K nearest neighbours using a KDTree
    
    pts = np.array(list(zip(np.nonzero(gt)[1].ravel(), np.nonzero(gt)[0].ravel())))
    leafsize = 2048
    
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    
    # query kdtree
    distances, locations = tree.query(pts, k=4)

        
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        
        #Convolve with the gaussian filter
        
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    
    return density

In [21]:
get_new_batch(target=r'CompleteAnnotations_2016-07-11')

CompleteAnnotations_2016-07-11/BAILa.json
CompleteAnnotations_2016-07-11/BAILa.mat
CompleteAnnotations_2016-07-11/DAMOa.json
CompleteAnnotations_2016-07-11/DAMOa.mat
CompleteAnnotations_2016-07-11/GEORa.json
CompleteAnnotations_2016-07-11/GEORa.mat
CompleteAnnotations_2016-07-11/HALFb.json
CompleteAnnotations_2016-07-11/HALFb.mat
CompleteAnnotations_2016-07-11/HALFc.json
CompleteAnnotations_2016-07-11/HALFc.mat
CompleteAnnotations_2016-07-11/LOCKb.json
CompleteAnnotations_2016-07-11/LOCKb.mat
CompleteAnnotations_2016-07-11/MAIVb.json
CompleteAnnotations_2016-07-11/MAIVb.mat
CompleteAnnotations_2016-07-11/MAIVc.json
CompleteAnnotations_2016-07-11/MAIVc.mat
CompleteAnnotations_2016-07-11/NEKOa.json
CompleteAnnotations_2016-07-11/NEKOa.mat
CompleteAnnotations_2016-07-11/NEKOb.json
CompleteAnnotations_2016-07-11/NEKOb.mat
CompleteAnnotations_2016-07-11/NEKOc.json
CompleteAnnotations_2016-07-11/NEKOc.mat
CompleteAnnotations_2016-07-11/PETEc.json
CompleteAnnotations_2016-07-11/PETEc.mat
Comp

In [22]:
# TODO BAILa will be from looping over master list
get_new_batch('BAILa')
df = run(filepath_=r'/content/CompleteAnnotations_2016-07-11/BAILa.json')
df.head()

Streaming output truncated to the last 5000 lines.
BAILa/BAILa2014a_000215.JPG
BAILa/BAILa2014a_000216.JPG
BAILa/BAILa2014a_000217.JPG
BAILa/BAILa2014a_000218.JPG
BAILa/BAILa2014a_000219.JPG
BAILa/BAILa2014a_000220.JPG
BAILa/BAILa2014a_000221.JPG
BAILa/BAILa2014a_000222.JPG
BAILa/BAILa2014a_000223.JPG
BAILa/BAILa2014a_000224.JPG
BAILa/BAILa2014a_000225.JPG
BAILa/BAILa2014a_000226.JPG
BAILa/BAILa2014a_000227.JPG
BAILa/BAILa2014a_000228.JPG
BAILa/BAILa2014a_000229.JPG
BAILa/BAILa2014a_000230.JPG
BAILa/BAILa2014a_000231.JPG
BAILa/BAILa2014a_000232.JPG
BAILa/BAILa2014a_000233.JPG
BAILa/BAILa2014a_000234.JPG
BAILa/BAILa2014a_000235.JPG
BAILa/BAILa2014a_000236.JPG
BAILa/BAILa2014a_000237.JPG
BAILa/BAILa2014a_000238.JPG
BAILa/BAILa2014a_000239.JPG
BAILa/BAILa2014a_000240.JPG
BAILa/BAILa2014a_000241.JPG
BAILa/BAILa2014a_000242.JPG
BAILa/BAILa2014a_000243.JPG
BAILa/BAILa2014a_000244.JPG
BAILa/BAILa2014a_000245.JPG
BAILa/BAILa2014a_000246.JPG
BAILa/BAILa2014a_000247.JPG
BAILa/BAILa2014a_000248.J

,imName,xy
0,BAILa2014a_000001,"[[[350, 1455], [849, 1327], [1129, 1042], [73,..."
1,BAILa2014a_000002,[]
2,BAILa2014a_000003,"[[[48, 293], [62, 293], [46, 319], [71, 314], ..."
3,BAILa2014a_000004,"[[[1340, 1140], [914, 1335], [192, 1162], [35,..."
4,BAILa2014a_000005,"[[[1022, 1309], [161, 1167], [23, 875], [530, ..."


In [29]:
os.chdir(VM_ROOT + '/BAILa')
for i, row in df.iterrows():
    if i > 0:
        break # TODO
    img_name = row['imName']
    img_dots = row['xy']
    img = image.imread(os.path.join('/content/BAILa', row['imName']) + '.JPG')

    dots = get_longest_dot_list(img_dots)
    k = make_sparse_mat(img.shape, dots)
    k = gaussian_filter_density(k)

    with h5py.File(img_name+'_gt.h5', 'w') as f:
        f['density'] = k